<a href="https://colab.research.google.com/github/h0ssn1/elasticsearch_bm25/blob/master/elasticsearch_pre_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code snippet I provided assumes a specific format for the output CSV file that is compatible with Elasticsearch.

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install Elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.9 MB/s eta 0:00:00


save in csv format

In [ ]:
import os
import csv
import hazm

# Set up the Hazm Persian text preprocessing library
normalizer = hazm.Normalizer()
tokenizer = hazm.WordTokenizer()
stemmer = hazm.Stemmer()
stopwords = hazm.stopwords_list()

data_dir = "/content/drive/MyDrive/G2-text/persian_limitted"

output_file = "/content/drive/MyDrive/G2-text/persian_limitted/preprocessed_data.csv"


with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Loop over the text files in the directory
    for filename in os.listdir(data_dir):
        if filename.endswith(".txt"):
            # Read in the text file
            with open(os.path.join(data_dir, filename), 'r') as f:
                text = f.read()


            text = normalizer.normalize(text)
            tokens = tokenizer.tokenize(text)
            filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords]

            # Join the filtered tokens into a single string
            preprocessed_text = " ".join(filtered_tokens)

            #output CSV file
            writer.writerow([filename, preprocessed_text])

save in json format

In [ ]:
import os
import json
import hazm

# hazm
normalizer = hazm.Normalizer()
tokenizer = hazm.WordTokenizer()
stemmer = hazm.Stemmer()
stopwords = hazm.stopwords_list()

#out and in
data_dir = "/content/drive/MyDrive/G2-text/persian_limitted"
output_dir = "/content/drive/MyDrive/G2-text/persian_limitted/preprocessed_data"
###################################################################################

os.makedirs(output_dir, exist_ok=True)

# Loop over the text files
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        # Read in the text file
        with open(os.path.join(data_dir, filename), 'r') as f:
            text = f.read()

        text = normalizer.normalize(text)
        tokens = tokenizer.tokenize(text)
        filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords]


        preprocessed_text = " ".join(filtered_tokens)

        # Create a dictionary representing the preprocessed data for this file
        preprocessed_dict = {
            "filename": filename,
            "preprocessed_text": preprocessed_text
        }

        # json output
        output_file = os.path.join(output_dir, os.path.splitext(filename)[0] + ".json")

        # Write the preprocessed data to the output JSON file
        with open(output_file, 'w') as f:
            json.dump(preprocessed_dict, f)

run In csv format and elastic search.

In [ ]:
from elasticsearch import Elasticsearch
import csv

#Elasticsearch set up
es = Elasticsearch(hosts=["http://94.130.59.207:9200/"])


input_file = "/content/drive/MyDrive/G2-text/persian_limitted/preprocessed_data.csv"

# Define the Elasticsearch indexing
mapping = {
    "mappings": {
        "properties": {
            "title": {
                "type": "keyword"
            },
            "body": {
                "type": "text",
                "analyzer": "persian"
            }
        }
    }
}

# Create the Elasticsearch index with the defined mapping
es.indices.create(index="my_index", body=mapping)

with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile)

    # Loop over the rows in the input file and index each document in Elasticsearch
    for row in reader:
        # Extract the document ID and preprocessed text from the row
        doc_id = row[0]
        preprocessed_text = row[1]

        # Create a JSON object representing the document
        doc = {
            "title": doc_id,
            "body": preprocessed_text
        }

        # Index the document in Elasticsearch
        es.index(index="my_index", id=doc_id, body=doc)

<ipython-input-22-76983ffece88>:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="my_index", body=mapping)


BadRequestError: ignored

In [ ]:
import hazm
from elasticsearch import Elasticsearch


normalizer = hazm.Normalizer()
tokenizer = hazm.WordTokenizer()
stemmer = hazm.Stemmer()
stopwords = hazm.stopwords_list()

question = "من گشنه هستم غذا چی بخورم"
normalized_question = normalizer.normalize(question)
tokenized_question = tokenizer.tokenize(normalized_question)
filtered_question_tokens = [stemmer.stem(token) for token in tokenized_question if token not in stopwords]
preprocessed_question = " ".join(filtered_question_tokens)


es = Elasticsearch("http://94.130.59.207:9200/")
index_name = "my_index"

# Construct the search request using the preprocessed question and BM25 scoring
search_body = {
    "query": {
        "match": {
            "body": {
                "query": preprocessed_question,
                "analyzer": "persian",
                "boost": 1.0
            }
        }
    },
    "explain": True
}

# Send the search request to Elasticsearch and retrieve the search results
response = es.search(index=index_name, body=search_body)

# Extract and analyze the search results
search_hits = response["hits"]["hits"]

for hit in search_hits:
    document_id = hit["_id"]
    document_score = hit["_score"]
    document_source = hit["_source"]
    print(f"Document ID: {document_id}, Score: {document_score}, Source: {document_source}")

<ipython-input-5-d4df5cf06be4>:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_body)


Document ID: ۱۰ اپلیکیشن مورد نیاز برای سفرهای خارجی.txt, Score: 2.052128, Source: {'title': '۱۰ اپلیکیشن مورد نیاز برای سفرهای خارجی.txt', 'body': 'امروزه پیشرف تکنولوژ کار وابسته اینترن اپلیکیش\u200cهاس . برنامه زندگ راح کرده\u200cاند . تقریبا کار نرم\u200cافزار ساخته\u200cشده ، مقوله سفر گردشگر قضیه مستثنا . به\u200cراح کلیک استفاده اپلیکیشن مختلف می\u200cتوانید بلیط بخرید ، هتل انتخاب رزرو مک دیدن کشور مقصد آشنا شوید به\u200cطور سفر برنامه\u200cریز . علاقه\u200cمند مهاجر خارج کشور هستید ، موسسه GO ۲ TR پروسه یار . می\u200cتوانید تماس تلفن رایگ ساع شبانه\u200cروز رزرو وق مشاوره تخصص راهکار راهنما مشاور بهره\u200cمند شوید . مقاله قصد کاربرد اپلیک سفر آشنا ، پا مقاله همراه . کاربرد اپلیکیشن سفر زمانی\u200cکه کشور شهر سفر می\u200cکنید نیازمند اطلاع : رستور ، مک دیدن ، وسایل نقلیه عمو . هستید ، اپلیکیشن مختلف رسیده\u200cاند سفر برا راح لذ بخ کرده\u200cاند . ادامه قصد اپلیکیشن سفر آشنا . تراول هیستور سابقه سفر اه سفر استون سفر پاراگوئه سفر یون سفر کامرون سفر ساحل\u200cعاج سفر روسیه سفر ا

<ipython-input-5-d4df5cf06be4>:36: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index=index_name, body=search_body)


ارزش امتیاز به این شکل مشخخص میشه که باید امتیاز بالاترین داکیومنت از mean مجموع داکیومنت ها بیتشر باشه

مثلا اینجا امتیاز 2.05 اصلا ارزشی نداره جون از mean باید خیلی بالاتر باشه

![Screenshot (372).png](data:image/png;base64,iVBORw0KGgoAAAANSUh

```
# This is formatted as

```
# This is formatted as code
```

 code
```

...)